In [77]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sklearn.feature_extraction.text as sktext
import sklearn.neighbors as skneigh
import scipy

In [49]:
def attr_parser(item_attrs, cat_attrs):
    # Parsea los atributes preservando orden
    parsed = []
    for base_attr in cat_attrs:
        attr_id = base_attr['id'] 

        # Look for attribute
        attribute_found = False
        for attr in item_attrs:
            if attr['id'] == attr_id:
                value_type = base_attr['value_type']
                try:
                    if   value_type == 'number':
                        parsed.append(int(attr['value_name']))
                    elif value_type == 'number_unit':
                        # TODO: conversor de unidades
                        parsed.append(attr['value_struct']['number']) 
                    elif value_type == 'string':
                        # TODO: apply tokenizer to attribute
                        parsed.append(attr['value_id'])
                    elif value_type == 'boolean':
                        parsed.append(True if attr['value_id']=='242085' else False)
                    elif value_type == 'list':
                        # TODO: implement list parsing
                        pass
                except TypeError:
                    parsed.append(None)
                
                attribute_found = True
                break

        # Attribute not found
        if not attribute_found:
            parsed.append(None)
    
    return parsed

In [19]:
# Target
site_id = 'MLA'
item_id = f'{site_id}1100783124'

In [70]:
# 1 - Get item
# Extract info from target item
url  = f'https://api.mercadolibre.com/items/{item_id}'
req  = requests.get(url)
info = req.json()
req  = requests.get(url+'/description')
desc = req.json()['plain_text']
target = info['title']

Obtengo la categoría y busco sus atributos. Para expandir el universo de posibilidades una opción podría ser buscar en que otras categorías se podría encontrar el item mediante la funcioón de _domain discovery_ de la API

In [16]:
# 2 - Get categoy and attributes
cat_id = info['category_id']
url = f'https://api.mercadolibre.com/categories/{cat_id}/attributes'
req = requests.get(url)
cat_attrs = req.json()

In [18]:
# 3 - Get items in categories
limit = 50
items = []

url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={cat_id}'
req = requests.get(url)
total = req.json()['paging']['total']
if total>1000: total = 1000
    
total = 200 ##### ACTUALIZAR ESTO DESPUES

items = items + req.json()['results']
for offset in range(limit, total, limit):
    url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={cat_id}&offset={offset}'
    req = requests.get(url)
    items = items + req.json()['results']

In [ ]:
# Extract features from items
titles = []
attributes  = []
for unit in items:
    url  = f'https://api.mercadolibre.com/items/{unit["id"]}'
    req  = requests.get(url)
    info = req.json()
    titles.append(info['title'])
    attributes.append(attr_parser(info['attributes'], cat_attrs))

    # Get descriptions at random to help build vocabulary
    if False:
        req  = requests.get(url+'/description')
        desc = req.json()['plain_text']

In [72]:
count_vect = sktext.CountVectorizer()
tfidf_tran = sktext.TfidfTransformer()

X = count_vect.fit_transform(titles)
X = tfidf_tran.fit_transform(X)

In [73]:
nnalg = skneigh.NearestNeighbors()
nnalg.fit(X)

NearestNeighbors()

In [78]:
ll=[]

predict = count_vect.transform([target])
X_tfidf2 = tfidf_tran.transform(predict)
ll.append(X_tfidf2)

ll = scipy.sparse.vstack((ll))
res = nnalg.kneighbors(ll, return_distance=False)

In [79]:
res

array([[  0, 145,   4,  15,  91]])